In [8]:
import numpy as np
from scipy.spatial.distance import pdist, squareform

def generate_distance_matrix_format(num_nodes, seed, file_name):
    """
    Generate a random symmetric distance matrix for the TSP problem, print it in the required format,
    and save it as a .npy file.
    
    Parameters:
    num_nodes (int): Number of nodes (cities) in the matrix.
    seed (int): Seed value for random number generator.
    file_name (str): The name of the file to save the numpy array.
    
    Returns:
    None
    """
    np.random.seed(seed)
    # Generate random coordinates for cities
    coordinates = np.random.rand(num_nodes, 2) * 100
    # Create distance matrix based on Euclidean distance
    distance_matrix = squareform(pdist(coordinates))
    
    # Save the distance matrix as a .npy file
    np.save(file_name, distance_matrix)
    
    # Format the matrix output as shown in the desired format
    print("D= [", end="")
    for i, row in enumerate(distance_matrix):
        if i == 0:
            print("[", end="")
        else:
            print(" [", end="")
        print(" ".join([f"{value:.4f}" for value in row]), end="]")
        if i != len(distance_matrix) - 1:
            print(" ")
    print("]")

# Example usage: generate a matrix for 20 nodes with seed 4 and save it as 'distance_matrix.npy'
generate_distance_matrix_format(20, 4, 'distance_matrix.npy')


D= [[0.0000 16.7679 42.6824 54.1081 72.2846 39.6730 44.8404 80.4749 97.1422 100.9611 66.5760 36.6686 92.3728 52.7261 27.1762 58.7530 62.5362 80.8809 45.1759 57.4761] 
 [16.7679 0.0000 56.9499 70.8595 77.2259 55.2078 29.0120 81.7330 101.8066 95.9501 58.5409 24.0169 102.4041 66.1346 38.8755 49.7439 75.5898 83.1935 45.4880 57.4877] 
 [42.6824 56.9499 0.0000 34.8753 49.5611 8.3703 78.4907 65.6035 70.9528 98.7718 77.8171 65.6206 54.3220 10.8266 19.5630 74.1199 20.6468 63.1966 45.5764 52.5227] 
 [54.1081 70.8595 34.8753 0.0000 84.0724 27.4629 98.3715 100.4714 103.9378 133.1218 108.6513 88.0789 80.6044 40.8882 46.9865 103.3648 46.5873 97.9642 77.6420 86.2879] 
 [72.2846 77.2259 49.5611 84.0724 0.0000 57.7336 82.0418 18.5381 24.8715 56.2091 54.5835 68.6021 36.8589 43.8326 48.1120 57.3038 42.2934 14.5499 33.8829 26.1318] 
 [39.6730 55.2078 8.3703 27.4629 57.7336 0.0000 79.0152 73.3900 79.3215 105.6686 82.5985 66.8640 61.8518 18.1094 21.5721 78.1455 27.2784 71.1764 50.8477 58.9247] 
 [44.8404 29

In [16]:
import random
import numpy as np

# Set random seeds for reproducibility
random_seed = 1000
numpy_seed = 1000

# Set seed for Python's random module
random.seed(random_seed)

# Set seed for NumPy's random module
np.random.seed(numpy_seed)

# Sample data: 5 cities with a symmetric distance matrix (TSP instance)
cities = [0, 1, 2, 3, 4]
distance_matrix = np.array([
    [0, 2, 9, 10, 7],
    [2, 0, 6, 4, 3],
    [9, 6, 0, 8, 5],
    [10, 4, 8, 0, 6],
    [7, 3, 5, 6, 0]
])

# Utility functions from the algorithm
def calculate_cost(route, distance_matrix):
    return sum(distance_matrix[route[i], route[(i + 1) % len(route)]] for i in range(len(route)))

def construct_randomized_greedy_tour(cities, distance_matrix, k=3):
    tour = [random.choice(cities)]
    unvisited = set(cities) - set(tour)
    while unvisited:
        current = tour[-1]
        neighbors = sorted(unvisited, key=lambda x: distance_matrix[current][x])[:k]
        next_city = random.choice(neighbors)
        tour.append(next_city)
        unvisited.remove(next_city)
    return tour

def swap_cost(route, i, j, distance_matrix):
    A, B = route[i], route[i + 1]
    C, D = route[j], route[(j + 1) % len(route)]
    return (distance_matrix[A][C] + distance_matrix[B][D] -
            distance_matrix[A][B] - distance_matrix[C][D])

# def two_opt(route, distance_matrix):
#     improved = True
#     while improved:
 
#         improved = False
#         for i in range(len(route) - 1):
#             for j in range(i + 2, len(route)):
#                 if j - i == 1: continue  # Skip adjacent nodes
#                 if swap_cost(route, i, j, distance_matrix) < 0:
#                     route[i+1:j+1] = reversed(route[i+1:j+1])
#                     improved = True
#     return route


def two_opt(route, distance_matrix, max_swaps=50):
    improved = True
    swap_count = 0
    while improved and swap_count < max_swaps:
        improved = False
        for i in range(len(route) - 1):
            for j in range(i + 2, len(route)):
                if j - i == 1: continue  # Skip adjacent nodes
                if swap_cost(route, i, j, distance_matrix) < 0:
                    route[i+1:j+1] = reversed(route[i+1:j+1])
                    improved = True
                    swap_count += 1
                    if swap_count >= max_swaps:
                        return route
    return route

def perturb_route(route, factor):
    size = len(route)
    i, j = random.sample(range(size), 2)
    i, j = min(i, j), max(i, j)
    if j - i <= factor:
        route[i:j+1] = reversed(route[i:j+1])
    else:
        random.shuffle(route[i:j+1])
    return route

def AGRP_TSP(cities, distance_matrix, I_max=4, S=2, P=2):
    best_route = None
    best_cost = float('inf')
    stagnation_counter = 0
    perturbation_factor = P

    for iteration in range(I_max):
        print('iteration', iteration)
        # Greedy Randomized Construction
        route = construct_randomized_greedy_tour(cities, distance_matrix, k=3)
        route = two_opt(route, distance_matrix)  # Local optimization

        # Update Best Solution
        current_cost = calculate_cost(route, distance_matrix)
        if current_cost < best_cost:
            best_route, best_cost = route, current_cost
            stagnation_counter = 0  # Reset stagnation counter
            perturbation_factor = P  # Reset perturbation factor
        else:
            stagnation_counter += 1

        # Apply Perturbation if Stagnating
        if stagnation_counter >= S:
            route = perturb_route(best_route, factor=perturbation_factor)
            perturbation_factor += 1  # Increase perturbation factor
            stagnation_counter = 0  # Reset stagnation counter

    return best_route, best_cost


cities= [0,1,2,3,4,5,6]
distance_matrix =np.array(  [[0.0,4.3092,3.8329,2.7762,1.7441,4.2076,6.0199]
,[4.3092,0.,1.6596,3.7435,5.6408,2.5764,7.5691]
,[3.8329,1.6596,0.,2.3055,4.7278,3.8966,8.4056]
,[2.7762,3.7435,2.3055,0.,2.8311,5.2142,8.5694]
,[1.7441,5.6408,4.7278,2.8311,0.,5.9246,7.3483]
,[4.2076,2.5764,3.8966,5.2142,5.9246,0.,5.1733]
,[6.0199,7.5691,8.4056,8.5694,7.3483,5.1733,0.0]])



# Run the heuristic on the sample data
best_route, best_cost = AGRP_TSP(cities, distance_matrix)
print (best_cost)


iteration 0
iteration 1
iteration 2
iteration 3
22.3099


In [42]:
import random
import numpy as np
import time

# Sample data: 7 cities with a symmetric distance matrix (TSP instance)
cities = [0, 1, 2, 3, 4, 5, 6]
distance_matrix=np.array(
    [[0.0000,50.1714,82.4215,32.7554,33.1981,35.4476,86.8834,79.1141,43.1720,66.1979,84.5219,59.0133,18.4527,47.0205,93.0141,81.4241,30.9494,60.7983,85.5903,59.7146]
,[50.1714,0.0000,72.6429,72.5066,17.0589,80.2453,39.9165,68.9291,43.4089,42.9584,47.4622,49.7422,33.3051,53.8973,56.9575,44.5855,75.3307,31.3970,65.8514,16.5545]
,[82.4215,72.6429,0.0000,71.6903,70.9155,82.5106,67.8767,3.7647,39.7436,30.7285,45.6133,25.0569,69.7065,36.0890,44.9935,45.1760,79.8092,103.9761,15.9631,59.9797]
,[32.7554,72.5066,71.6903,0.0000,56.5579,11.0304,101.3477,69.3894,42.0906,68.5811,91.3634,55.1905,40.6146,37.9356,97.8499,88.6476,8.3002,90.9386,80.7078,75.7337]
,[33.1981,17.0589,70.9155,56.5579,0.0000,63.6847,54.7050,67.1625,34.8910,44.9863,56.8680,46.0683,16.4265,44.5790,66.1679,53.7587,58.7669,37.8071,67.8603,28.0517]
,[35.4476,80.2453,82.5106,11.0304,63.6847,0.0000,111.0418,80.2863,52.7848,79.3615,101.9406,66.1827,47.3048,48.9605,108.6171,99.1736,4.9187,95.8750,91.7317,84.9160]
,[86.8834,39.9165,67.8767,101.3477,54.7050,111.0418,0.0000,65.1261,61.3678,40.8194,23.2306,56.1823,68.5418,70.1097,29.1775,22.8752,106.3668,61.0758,54.0141,27.1781]
,[79.1141,68.9291,3.7647,69.3894,67.1625,80.2863,65.1261,0.0000,36.2639,27.2371,43.2264,21.3644,66.1316,33.1311,43.2828,42.5915,77.4337,100.2482,14.9757,56.4210]
,[43.1720,43.4089,39.7436,42.0906,34.8910,52.7848,61.3678,36.2639,0.0000,26.6163,49.3003,15.8856,30.1447,10.5097,55.8408,46.6340,48.6927,71.9645,42.7084,38.9295]
,[66.1979,42.9584,30.7285,68.5811,44.9863,79.3615,40.8194,27.2371,26.6163,0.0000,23.5352,15.8711,49.4536,32.1996,29.2689,21.3084,75.3076,74.3159,23.1684,29.2963]
,[84.5219,47.4622,45.6133,91.3634,56.8680,101.9406,23.2306,43.2264,49.3003,23.5352,0.0000,39.2024,66.4209,55.7066,9.5052,3.1328,97.6784,75.6698,31.0072,30.9248]
,[59.0133,49.7422,25.0569,55.1905,46.0683,66.1827,56.1823,21.3644,15.8856,15.8711,39.2024,0.0000,44.9752,17.4405,43.8754,37.1254,62.5224,80.5593,26.8979,39.8972]
,[18.4527,33.3051,69.7065,40.6146,16.4265,47.3048,68.5418,66.1316,30.1447,49.4536,66.4209,44.9752,0.0000,37.3026,75.1203,63.3041,42.3906,51.0091,70.3989,41.3642]
,[47.0205,53.8973,36.0890,37.9356,44.5790,48.9605,70.1097,33.1311,10.5097,32.1996,55.7066,17.4405,37.3026,0.0000,61.0792,53.3381,45.5022,82.0502,42.9577,49.0190]
,[93.0141,56.9575,44.9935,97.8499,66.1679,108.6171,29.1775,43.2828,55.8408,29.2689,9.5052,43.8754,75.1203,61.0792,0.0000,12.4993,104.5228,84.8442,29.2323,40.4267]
,[81.4241,44.5855,45.1760,88.6476,53.7587,99.1736,22.8752,42.5915,46.6340,21.3084,3.1328,37.1254,63.3041,53.3381,12.4993,0.0000,94.8760,73.1511,31.1568,28.0322]
,[30.9494,75.3307,79.8092,8.3002,58.7669,4.9187,106.3668,77.4337,48.6927,75.3076,97.6784,62.5224,42.3906,45.5022,104.5228,94.8760,0.0000,91.1472,88.4282,80.1371]
,[60.7983,31.3970,103.9761,90.9386,37.8071,95.8750,61.0758,100.2482,71.9645,74.3159,75.6698,80.5593,51.0091,82.0502,84.8442,73.1511,91.1472,0.0000,97.0606,46.3454]
,[85.5903,65.8514,15.9631,80.7078,67.8603,91.7317,54.0141,14.9757,42.7084,23.1684,31.0072,26.8979,70.3989,42.9577,29.2323,31.1568,88.4282,97.0606,0.0000,51.1276]
,[59.7146,16.5545,59.9797,75.7337,28.0517,84.9160,27.1781,56.4210,38.9295,29.2963,30.9248,39.8972,41.3642,49.0190,40.4267,28.0322,80.1371,46.3454,51.1276,0.0000]]


)

# Utility functions from the algorithm
def calculate_cost(route, distance_matrix):
    return sum(distance_matrix[route[i], route[(i + 1) % len(route)]] for i in range(len(route)))

def construct_randomized_greedy_tour(cities, distance_matrix, k=3):
    tour = [random.choice(cities)]
    unvisited = set(cities) - set(tour)
    while unvisited:
        current = tour[-1]
        neighbors = sorted(unvisited, key=lambda x: distance_matrix[current][x])[:k]
        next_city = random.choice(neighbors)
        tour.append(next_city)
        unvisited.remove(next_city)
    return tour

def swap_cost(route, i, j, distance_matrix):
    A, B = route[i], route[i + 1]
    C, D = route[j], route[(j + 1) % len(route)]
    return (distance_matrix[A][C] + distance_matrix[B][D] -
            distance_matrix[A][B] - distance_matrix[C][D])

def two_opt_limited(route, distance_matrix, max_swaps=50):
    improved = True
    swap_count = 0
    while improved and swap_count < max_swaps:
        improved = False
        for i in range(len(route) - 1):
            for j in range(i + 2, len(route)):
                if j - i == 1: continue  # Skip adjacent nodes
                if swap_cost(route, i, j, distance_matrix) < 0:
                    route[i+1:j+1] = reversed(route[i+1:j+1])
                    improved = True
                    swap_count += 1
                    if swap_count >= max_swaps:
                        return route
    return route

def perturb_route(route, factor):
    size = len(route)
    i, j = random.sample(range(size), 2)
    i, j = min(i, j), max(i, j)
    if j - i <= factor:
        route[i:j+1] = reversed(route[i:j+1])
    else:
        random.shuffle(route[i:j+1])
    return route

def AGRP_TSP(cities, distance_matrix, I_max=10, S=2, P=2):
    best_route = None
    best_cost = float('inf')
    stagnation_counter = 0
    perturbation_factor = P

    for iteration in range(I_max):
        print(f"iteration {iteration}")
        route = construct_randomized_greedy_tour(cities, distance_matrix, k=3)
        route = two_opt_limited(route, distance_matrix, max_swaps=50)  # Local optimization

        # Update Best Solution
        current_cost = calculate_cost(route, distance_matrix)
        if current_cost < best_cost:
            best_route, best_cost = route, current_cost
            stagnation_counter = 0  # Reset stagnation counter
            perturbation_factor = P  # Reset perturbation factor
        else:
            stagnation_counter += 1

        # Apply Perturbation if Stagnating
        if stagnation_counter >= S:
            route = perturb_route(best_route, factor=perturbation_factor)
            perturbation_factor += 1  # Increase perturbation factor
            stagnation_counter = 0  # Reset stagnation counter

    return best_route, best_cost

# Main loop to try different seeds for 5 seconds each
timeout = 5  # seconds
found_solution = False


# seedd= 1000
# random.seed(seedd)
# np.random.seed(seedd)
best_route, best_cost = AGRP_TSP(cities, distance_matrix)
print(f"Solution found with seed and cost {best_cost}")



# for seed in range(10, 60):
#     random.seed(seed)
#     np.random.seed(seed)

#     start_time = time.time()

#     while time.time() - start_time < timeout:
#         best_route, best_cost = AGRP_TSP(cities, distance_matrix)
#         if best_cost is not None:  # Found a solution
#             found_solution = True
#             print(f"Solution found with seed {seed} and cost {best_cost}")
#             break

#     if found_solution:
#         break
#     else:
#         print(f"No solution found within {timeout} seconds for seed {seed}")


iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
Solution found with seed and cost 276.21849999999995


You are an expert in solving optimization problems. Given a
Traveling Salesman Problem (TSP), your task is to find the shortest
possible route that visits every city exactly once and returns to the
starting point. The distance matrix is given in D.Search directly with no explanation 


D = a symmetric distance matrix for the Traveling Salesman Problem (TSP) using 20 cities (nodes) and a random seed value of 42. The matrix should be based on the Euclidean distance between random city coordinates in a 2D plane, with each value formatted to 4 decimal places. The output should be displayed as a nested list, where each row is enclosed in square brackets and values are separated by spaces, with rows printed one after the other. The matrix should be enclosed in an overall bracket.

dont show the solution, direct solve